# 24 layers, pre-trained weight

In [1]:
import tensorflow as tf
import numpy as np
import time
import os
import re

from PIL import Image as PI

# from tensorflow.examples.tutorials.mnist import input_data
# mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

In [2]:
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess_ :
    saver = tf.train.import_meta_graph('D:/Backup_data/jongkeun/paper/190524/save_dir-494.meta')
    saver.restore(sess_, tf.train.latest_checkpoint('D:/Backup_data/jongkeun/paper/190524/'))
    
    graph = tf.get_default_graph()
    
#     print(graph.get_tensor_by_name('layer0_conv2d_1/kernel:0'))
    weight0_1, weight0_2 = sess_.run([graph.get_tensor_by_name('layer0_conv2d_1/kernel:0'), 
                                      graph.get_tensor_by_name('layer0_conv2d_2/kernel:0')])
    weight1, weight1_1, weight1_2 = sess_.run([graph.get_tensor_by_name('layer1_conv2d/kernel:0'), 
                                               graph.get_tensor_by_name('layer1_conv2d_1/kernel:0'),
                                               graph.get_tensor_by_name('layer1_conv2d_2/kernel:0')])
    weight2, weight2_1, weight2_2 = sess_.run([graph.get_tensor_by_name('layer2_conv2d/kernel:0'), 
                                               graph.get_tensor_by_name('layer2_conv2d_1/kernel:0'),
                                               graph.get_tensor_by_name('layer2_conv2d_2/kernel:0')])
    weight3, weight3_1, weight3_2 = sess_.run([graph.get_tensor_by_name('layer3_conv2d/kernel:0'), 
                                               graph.get_tensor_by_name('layer3_conv2d_1/kernel:0'),
                                               graph.get_tensor_by_name('layer3_conv2d_2/kernel:0')])
    
    
#     weight_ = weight[:,:,:,:]
#     print(weight_)

print("++++++++++++++++++ DONE ++++++++++++++++++")

W1019 00:42:50.610228 17456 deprecation.py:323] From C:\Users\whdrm\Anaconda3\lib\site-packages\tensorflow\python\training\saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


++++++++++++++++++ DONE ++++++++++++++++++


In [3]:
training_epochs = 500
batch_size = 20
image_resize = 128
image_width = image_resize
image_height = image_resize
initial_rate = 1e-5
label_len = 3

home_dir = "D:/Backup_data/jongkeun"

image_file = "/dir_images/190819_newTraining/"
test_image_file = "/dir_images/190309_new_test/"

label_file = "/csv/190819_newTraining.csv"
test_label_file = "/csv/190309_new_test.csv"

In [4]:
label_dir = home_dir + label_file
test_label_dir = home_dir + test_label_file

# label_dir = os.getcwd() + "/csv/181126.csv"
# test_label_dir = os.getcwd() + "/csv/181126_test.csv"

f_label = open(label_dir, 'r')
f_test_label = open(test_label_dir, 'r')

label_data = f_label.readlines()
test_label_data = f_test_label.readlines()

f_label.close()
f_test_label.close()

label_array = np.array(np.reshape(label_data, [-1, label_len]), dtype=np.int32)
test_label_array = np.array(np.reshape(test_label_data, [-1, label_len]), dtype=np.int32)

print(len(label_array))
print(len(test_label_array))

9182
1018


In [5]:
image_dir = home_dir + image_file
test_image_dir = home_dir + test_image_file

# image_dir = os.getcwd() + "/dir_images/181126/"
# test_image_dir = os.getcwd() + "/dir_images/181126_test/"

image_list = os.listdir(image_dir)
test_image_list = os.listdir(test_image_dir)

image_list = sorted(image_list, key=lambda x: (int(re.sub('\D','',x)),x))
test_image_list = sorted(test_image_list, key=lambda y: (int(re.sub('\D','',y)),y))

image_array = []
test_image_array = []

for i in range(len(image_list)) :
    image_list[i] = image_dir + image_list[i]
    image_array.append(np.array(PI.open(image_list[i]).resize((image_resize, image_resize)).convert('L')))

for j in range(len(test_image_list)) :
    test_image_list[j] = test_image_dir + test_image_list[j]
    test_image_array.append(np.array(PI.open(test_image_list[j]).resize((image_resize, image_resize)).convert('L')))

image_array = np.reshape(image_array, [-1, image_width*image_height])
test_image_array = np.reshape(test_image_array, [-1, image_width*image_height])

image_array = np.array(np.reshape(image_array, [-1, image_width, image_height, 1]), dtype=np.float32)
test_image_array = np.array(np.reshape(test_image_array, [-1, image_width, image_height, 1]), dtype=np.float32)

print(len(image_array))
print(len(test_image_array))

9182
1018


In [6]:
x = tf.placeholder(dtype=tf.float32, shape=[None, image_width, image_height, 1], name='input_image')
y_ = tf.placeholder(dtype=tf.float32, shape=[None, label_len], name='input_label')
learning_rate = tf.placeholder(dtype=tf.float32, name='learning_rate')

# W_pre_trained = tf.placeholder(dtype=tf.float32, shape=[3,3,8,128], name='pre_trained_weight')

is_training = tf.placeholder(dtype=tf.bool)

In [7]:
W_h0_1 = tf.placeholder(dtype=tf.float32, shape=[3,3,1,8], name='W_h0_1')
W_h0_2 = tf.placeholder(dtype=tf.float32, shape=[3,3,8,128], name='W_h0_2')

W_h1 = tf.placeholder(dtype=tf.float32, shape=[3,3,128,256], name='W_h1')
W_h1_1 = tf.placeholder(dtype=tf.float32, shape=[3,3,256, 128], name='W_h1_1')
W_h1_2 = tf.placeholder(dtype=tf.float32, shape=[3,3,128,256], name='W_h1_2')

W_h2 = tf.placeholder(dtype=tf.float32, shape=[3,3,256,512], name='W_h2')
W_h2_1 = tf.placeholder(dtype=tf.float32, shape=[3,3,512,256], name='W_h2_1')
W_h2_2 = tf.placeholder(dtype=tf.float32, shape=[3,3,256,512], name='W_h2_2')

W_h3 = tf.placeholder(dtype=tf.float32, shape=[3,3,512,512], name='W_h3')
W_h3_1 = tf.placeholder(dtype=tf.float32, shape=[3,3,512,1024], name='W_h3_1')
W_h3_2 = tf.placeholder(dtype=tf.float32, shape=[3,3,1024,512], name='W_h3_2')

In [8]:
def pooling(input_, dropout, is_training) :
    pooling = tf.layers.max_pooling2d(input_, [2, 2], [2, 2], padding='SAME')
    pooling = tf.layers.dropout(pooling, dropout, is_training)
    
    return pooling

def learning_layer(input_data, size, dropout, is_training) :
    layer = tf.layers.conv2d(input_data, size, [3, 3], padding='SAME')
    layer = tf.layers.max_pooling2d(layer, [2, 2], [2, 2], padding='SAME')
    layer = tf.layers.dropout(layer, dropout, is_training)
    
    return layer

def convolutional(input_, weight_) :
    conv_ = tf.nn.conv2d(input_, weight_, strides=[1,1,1,1], padding='SAME')
    hidden_ = tf.nn.relu(conv_)
    
    return hidden_

with tf.device('/gpu:0') :
    layer0_1 = convolutional(x, W_h0_1)
    layer0_2 = convolutional(layer0_1, W_h0_2)
    layer1 = convolutional(layer0_2, W_h1)
    layer1_ = pooling(layer1, dropout=0.7, is_training=is_training)
    
    layer1_1 = convolutional(layer1_, W_h1_1)
    layer1_2 = convolutional(layer1_1, W_h1_2)
    layer2 = convolutional(layer1_2, W_h2)
    layer2_ = pooling(layer2, dropout=0.7, is_training=is_training)
    
    layer2_1 = convolutional(layer2_, W_h2_1)
    layer2_2 = convolutional(layer2_1, W_h2_2)
    layer2_3 = convolutional(layer2_2, W_h3)
    layer2_4 = convolutional(layer2_3, W_h3_1)
    layer2_5 = convolutional(layer2_4, W_h3_2)
    layer2_6 = tf.layers.conv2d(layer2_5, 512, [3,3], padding='SAME')
    layer2_7 = tf.layers.conv2d(layer2_6, 256, [3,3], padding='SAME')
    layer3 = learning_layer(input_data=layer2_7, size=512, dropout=0.7, is_training=is_training)
    
    layer3_1 = tf.layers.conv2d(layer3, 1024, [3,3], padding='SAME')
    layer3_2 = tf.layers.conv2d(layer3_1, 256, [3,3], padding='SAME')
    layer3_3 = tf.layers.conv2d(layer3_2, 512, [3,3], padding='SAME')
    layer3_4 = tf.layers.conv2d(layer3_3, 128, [3,3], padding='SAME')
    layer3_5 = tf.layers.conv2d(layer3_4, 1024, [3,3], padding='SAME')
    layer3_6 = tf.layers.conv2d(layer3_5, 256, [3,3], padding='SAME')
    layer3_7 = tf.layers.conv2d(layer3_6, 512, [3,3], padding='SAME')
    layer3_8 = tf.layers.conv2d(layer3_7, 128, [3,3], padding='SAME')
    layer3_9 = tf.layers.conv2d(layer3_8, 1024, [3,3], padding='SAME')
    layer4 = learning_layer(input_data=layer3_9, size=1024, dropout=0.7, is_training=is_training)

W1019 00:46:38.940043 17456 deprecation.py:323] From <ipython-input-8-0b36f8f8bd5d>:2: max_pooling2d (from tensorflow.python.layers.pooling) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.MaxPooling2D instead.
W1019 00:46:39.123559 17456 deprecation.py:323] From <ipython-input-8-0b36f8f8bd5d>:3: dropout (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dropout instead.
W1019 00:46:39.200346 17456 deprecation.py:323] From <ipython-input-8-0b36f8f8bd5d>:36: conv2d (from tensorflow.python.layers.convolutional) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
W1019 00:46:39.201344 17456 deprecation.py:506] From C:\Users\whdrm\Anaconda3\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecat

In [9]:
fully_layer = tf.contrib.layers.flatten(layer4)
fully_layer = tf.layers.dense(fully_layer, 1024, activation=tf.nn.relu, name='fully_connected_layer')
fully_layer = tf.layers.dropout(fully_layer, 0.5, is_training, name='fully_connected_dropout')

fully_layer2 = tf.layers.dense(fully_layer, 512, activation=tf.nn.relu, name='fully_connected_layer2')
fully_layer2 = tf.layers.dropout(fully_layer2, 0.5, is_training, name='fully_connected_dropout2')

W1019 00:46:43.002177 17456 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W1019 00:46:43.004172 17456 deprecation.py:323] From C:\Users\whdrm\Anaconda3\lib\site-packages\tensorflow\contrib\layers\python\layers\layers.py:1634: flatten (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.flatten instead.
W1019 00:46:43.222588 17456 deprecation.py:323] From <ipython-input-9-f9981dd3a3a2>:2: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.


In [10]:
logits = tf.layers.dense(fully_layer2, label_len, activation=None)

with tf.name_scope('pred') : 
    pred = tf.nn.softmax(logits)
with tf.name_scope('loss') :
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y_))
    
with tf.device('/gpu:0') :
    with tf.name_scope('train') :
            train = tf.train.AdamOptimizer(learning_rate).minimize(loss)

correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y_, 1))
acc = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

label_output = tf.argmax(pred, 1)
label_input = tf.argmax(y_, 1)

In [11]:
config = tf.ConfigProto(log_device_placement = True, allow_soft_placement=True)
config.gpu_options.allow_growth = True

with tf.Session(config=config) as sess :
    coord = tf.train.Coordinator()
    thread = tf.train.start_queue_runners(sess=sess, coord=coord)
        
    sess.run(tf.global_variables_initializer())
    
    total_batch = int(len(image_list) / batch_size)
    correct_list = []
    incorrect_list = []
    cost_list = []
    cost_sum_list = []
    pred_list = []
    total_acc = 0
    cost_sum = 0
    cost_flag = 0
    rate_ = 1
    
    start_time = time.time()
    
    print("")
    
    for epoch in range(training_epochs) :
        total_cost = 0
        
        if epoch%50 == 49 :
                rate_ *= 0.8
                
        if epoch%10 == 9 :
#             rate_ *= 0.8
            midpoint = int(time.time() - start_time)
        
            print("")
            print("=======================================================================================")
            print('{:03d}:{:02d}:{:02d}'.format(midpoint//3600, (midpoint%3600//60), midpoint%60))
            print("=======================================================================================")
            print("")
        
        batch_index = np.random.choice(len(image_array), total_batch, replace=False)
                
        for i in range(total_batch) :                
            _, _loss = sess.run([train, loss], feed_dict={x: image_array[[batch_index[i]]], y_: label_array[[batch_index[i]]], 
                                                          is_training: True, learning_rate: initial_rate*rate_,
                                                          W_h0_1: weight0_1, W_h0_2: weight0_2,
                                                          W_h1: weight1, W_h1_1: weight1_1, W_h1_2: weight1_2,
                                                          W_h2: weight2, W_h2_1: weight2_1, W_h2_2: weight2_2,
                                                          W_h3: weight3, W_h3_1: weight3_1, W_h3_2: weight3_2})                 
            total_cost += _loss

        avg_cost = total_cost/total_batch
        print('Epoch : ', '%4d' % (epoch + 1), '    Avg. cost = ', '{:.4f}'.format(avg_cost))            
        
#         train_writer.add_summary(_merge, epoch)
               
    print("")
    print("=======================================================================================")
    print("================================     Training done     ================================")
    print("=======================================================================================")
    print("")
    
    for test in range(len(test_image_array)) :
        _acc, _label_input, _label_output, _pred = sess.run([acc, label_input, label_output, pred], 
                             feed_dict={x: test_image_array[[test]], y_: test_label_array[[test]], is_training: False,
                                        W_h0_1: weight0_1, W_h0_2: weight0_2,
                                        W_h1: weight1, W_h1_1: weight1_1, W_h1_2: weight1_2,
                                        W_h2: weight2, W_h2_1: weight2_1, W_h2_2: weight2_2,
                                        W_h3: weight3, W_h3_1: weight3_1, W_h3_2: weight3_2})

        total_acc += _acc
        pred_list.append(list(_pred[0]))
        
        print("n: ", test, "    label_input: ", _label_input, "    label_output: ", _label_output)
        
        if _label_input == _label_output :
            correct_list.append(test)   
        else :
            incorrect_list.append(test)
        
    print("")
    print("          TOTAL ACC    : ", '{:.5f}'.format(total_acc / len(test_image_array)))
    print("")
    print(pred_list)
    
    end_time = int(time.time() - start_time)
    
    print("")
    print("=======================================================================================")
    print('{:03d}:{:02d}:{:02d}'.format(end_time//3600, (end_time%3600//60), end_time%60))
    print("=======================================================================================")
    print("")
    
    coord.request_stop()
    coord.join(thread)

W1019 00:46:44.147114 17456 deprecation.py:323] From <ipython-input-11-4cb70f9ea11e>:6: start_queue_runners (from tensorflow.python.training.queue_runner_impl) is deprecated and will be removed in a future version.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
W1019 00:46:44.148111 17456 queue_runner_impl.py:471] `tf.train.start_queue_runners()` was called when no queue runners were defined. You can safely remove the call to this deprecated function.



Epoch :     1     Avg. cost =  0.8987
Epoch :     2     Avg. cost =  0.5945
Epoch :     3     Avg. cost =  0.6579
Epoch :     4     Avg. cost =  0.6152
Epoch :     5     Avg. cost =  0.5542
Epoch :     6     Avg. cost =  0.5616
Epoch :     7     Avg. cost =  0.5209
Epoch :     8     Avg. cost =  0.5475
Epoch :     9     Avg. cost =  0.5753

000:11:21

Epoch :    10     Avg. cost =  0.5225
Epoch :    11     Avg. cost =  0.4979
Epoch :    12     Avg. cost =  0.4579
Epoch :    13     Avg. cost =  0.5644
Epoch :    14     Avg. cost =  0.5364
Epoch :    15     Avg. cost =  0.4630
Epoch :    16     Avg. cost =  0.4933
Epoch :    17     Avg. cost =  0.5087
Epoch :    18     Avg. cost =  0.4546
Epoch :    19     Avg. cost =  0.4920

000:23:58

Epoch :    20     Avg. cost =  0.4475
Epoch :    21     Avg. cost =  0.4695
Epoch :    22     Avg. cost =  0.4844
Epoch :    23     Avg. cost =  0.4941
Epoch :    24     Avg. cost =  0.5086
Epoch :    25     Avg. cost =  0.4983
Epoch :    26     Avg. co

Epoch :   148     Avg. cost =  0.0703
Epoch :   149     Avg. cost =  0.0588

003:08:07

Epoch :   150     Avg. cost =  0.0571
Epoch :   151     Avg. cost =  0.0524
Epoch :   152     Avg. cost =  0.0478
Epoch :   153     Avg. cost =  0.0362
Epoch :   154     Avg. cost =  0.0227
Epoch :   155     Avg. cost =  0.0257
Epoch :   156     Avg. cost =  0.1046
Epoch :   157     Avg. cost =  0.0459
Epoch :   158     Avg. cost =  0.0393
Epoch :   159     Avg. cost =  0.0400

003:20:44

Epoch :   160     Avg. cost =  0.0809
Epoch :   161     Avg. cost =  0.0319
Epoch :   162     Avg. cost =  0.0418
Epoch :   163     Avg. cost =  0.0359
Epoch :   164     Avg. cost =  0.0542
Epoch :   165     Avg. cost =  0.0475
Epoch :   166     Avg. cost =  0.0329
Epoch :   167     Avg. cost =  0.0243
Epoch :   168     Avg. cost =  0.0285
Epoch :   169     Avg. cost =  0.0956

003:33:22

Epoch :   170     Avg. cost =  0.0635
Epoch :   171     Avg. cost =  0.0324
Epoch :   172     Avg. cost =  0.0466
Epoch :   173 

Epoch :   290     Avg. cost =  0.0158
Epoch :   291     Avg. cost =  0.0016
Epoch :   292     Avg. cost =  0.0014
Epoch :   293     Avg. cost =  0.0016
Epoch :   294     Avg. cost =  0.0137
Epoch :   295     Avg. cost =  0.0142
Epoch :   296     Avg. cost =  0.0147
Epoch :   297     Avg. cost =  0.0008
Epoch :   298     Avg. cost =  0.0012
Epoch :   299     Avg. cost =  0.0055

006:17:24

Epoch :   300     Avg. cost =  0.0083
Epoch :   301     Avg. cost =  0.0022
Epoch :   302     Avg. cost =  0.0014
Epoch :   303     Avg. cost =  0.0013
Epoch :   304     Avg. cost =  0.0101
Epoch :   305     Avg. cost =  0.0006
Epoch :   306     Avg. cost =  0.0020
Epoch :   307     Avg. cost =  0.0024
Epoch :   308     Avg. cost =  0.0010
Epoch :   309     Avg. cost =  0.0098

006:30:00

Epoch :   310     Avg. cost =  0.0037
Epoch :   311     Avg. cost =  0.0004
Epoch :   312     Avg. cost =  0.0003
Epoch :   313     Avg. cost =  0.0004
Epoch :   314     Avg. cost =  0.0001
Epoch :   315     Avg. cos

Epoch :   437     Avg. cost =  0.0000
Epoch :   438     Avg. cost =  0.0000
Epoch :   439     Avg. cost =  0.0000

009:13:59

Epoch :   440     Avg. cost =  0.0000
Epoch :   441     Avg. cost =  0.0000
Epoch :   442     Avg. cost =  0.0000
Epoch :   443     Avg. cost =  0.0038
Epoch :   444     Avg. cost =  0.0001
Epoch :   445     Avg. cost =  0.0001
Epoch :   446     Avg. cost =  0.0000
Epoch :   447     Avg. cost =  0.0000
Epoch :   448     Avg. cost =  0.0000
Epoch :   449     Avg. cost =  0.0000

009:26:36

Epoch :   450     Avg. cost =  0.0000
Epoch :   451     Avg. cost =  0.0000
Epoch :   452     Avg. cost =  0.0000
Epoch :   453     Avg. cost =  0.0000
Epoch :   454     Avg. cost =  0.0000
Epoch :   455     Avg. cost =  0.0000
Epoch :   456     Avg. cost =  0.0004
Epoch :   457     Avg. cost =  0.0000
Epoch :   458     Avg. cost =  0.0000
Epoch :   459     Avg. cost =  0.0001

009:39:12

Epoch :   460     Avg. cost =  0.0000
Epoch :   461     Avg. cost =  0.0000
Epoch :   462 

...

n:  392     label_input:  [1]     label_output:  [0]
n:  393     label_input:  [1]     label_output:  [1]
n:  394     label_input:  [1]     label_output:  [1]
n:  395     label_input:  [1]     label_output:  [1]
n:  396     label_input:  [1]     label_output:  [1]
n:  397     label_input:  [1]     label_output:  [1]
n:  398     label_input:  [1]     label_output:  [1]
n:  399     label_input:  [1]     label_output:  [1]
n:  400     label_input:  [1]     label_output:  [1]
n:  401     label_input:  [1]     label_output:  [1]
n:  402     label_input:  [1]     label_output:  [1]
n:  403     label_input:  [1]     label_output:  [1]
n:  404     label_input:  [1]     label_output:  [1]
n:  405     label_input:  [1]     label_output:  [1]
n:  406     label_input:  [1]     label_output:  [1]
n:  407     label_input:  [1]     label_output:  [1]
n:  408     label_input:  [1]     label_output:  [1]
n:  409     label_input:  [1]     label_output:  [1]
n:  410     label_input:  [1]     label_output

n:  548     label_input:  [1]     label_output:  [1]
n:  549     label_input:  [1]     label_output:  [0]
n:  550     label_input:  [1]     label_output:  [1]
n:  551     label_input:  [1]     label_output:  [2]
n:  552     label_input:  [1]     label_output:  [0]
n:  553     label_input:  [1]     label_output:  [1]
n:  554     label_input:  [1]     label_output:  [1]
n:  555     label_input:  [1]     label_output:  [1]
n:  556     label_input:  [1]     label_output:  [1]
n:  557     label_input:  [1]     label_output:  [1]
n:  558     label_input:  [1]     label_output:  [1]
n:  559     label_input:  [1]     label_output:  [1]
n:  560     label_input:  [1]     label_output:  [0]
n:  561     label_input:  [1]     label_output:  [1]
n:  562     label_input:  [1]     label_output:  [1]
n:  563     label_input:  [1]     label_output:  [1]
n:  564     label_input:  [1]     label_output:  [1]
n:  565     label_input:  [1]     label_output:  [1]
n:  566     label_input:  [1]     label_output

n:  703     label_input:  [2]     label_output:  [2]
n:  704     label_input:  [2]     label_output:  [2]
n:  705     label_input:  [2]     label_output:  [2]
n:  706     label_input:  [2]     label_output:  [2]
n:  707     label_input:  [2]     label_output:  [2]
n:  708     label_input:  [2]     label_output:  [2]
n:  709     label_input:  [2]     label_output:  [2]
n:  710     label_input:  [2]     label_output:  [2]
n:  711     label_input:  [2]     label_output:  [0]
n:  712     label_input:  [2]     label_output:  [2]
n:  713     label_input:  [2]     label_output:  [2]
n:  714     label_input:  [2]     label_output:  [2]
n:  715     label_input:  [2]     label_output:  [2]
n:  716     label_input:  [2]     label_output:  [0]
n:  717     label_input:  [2]     label_output:  [2]
n:  718     label_input:  [2]     label_output:  [2]
n:  719     label_input:  [2]     label_output:  [2]
n:  720     label_input:  [2]     label_output:  [2]
n:  721     label_input:  [2]     label_output

n:  859     label_input:  [2]     label_output:  [2]
n:  860     label_input:  [2]     label_output:  [1]
n:  861     label_input:  [2]     label_output:  [2]
n:  862     label_input:  [2]     label_output:  [2]
n:  863     label_input:  [2]     label_output:  [2]
n:  864     label_input:  [2]     label_output:  [2]
n:  865     label_input:  [2]     label_output:  [2]
n:  866     label_input:  [2]     label_output:  [0]
n:  867     label_input:  [2]     label_output:  [2]
n:  868     label_input:  [2]     label_output:  [2]
n:  869     label_input:  [2]     label_output:  [2]
n:  870     label_input:  [2]     label_output:  [2]
n:  871     label_input:  [2]     label_output:  [1]
n:  872     label_input:  [2]     label_output:  [2]
n:  873     label_input:  [2]     label_output:  [2]
n:  874     label_input:  [2]     label_output:  [2]
n:  875     label_input:  [2]     label_output:  [1]
n:  876     label_input:  [2]     label_output:  [1]
n:  877     label_input:  [2]     label_output

n:  1015     label_input:  [2]     label_output:  [2]
n:  1016     label_input:  [2]     label_output:  [2]
n:  1017     label_input:  [2]     label_output:  [2]

          TOTAL ACC    :  0.88114

[[0.9999995, 1.7728917e-08, 4.935775e-07], [0.999752, 0.00024805503, 2.37583e-11], [1.0, 1.7838992e-20, 9.739072e-15], [1.0, 5.363767e-26, 6.264083e-28], [1.0, 3.3415572e-21, 1.6670417e-16], [1.0, 6.2922186e-22, 4.6582948e-17], [0.9978149, 3.2488023e-10, 0.0021851084], [1.0, 1.44433285e-33, 7.298459e-30], [0.99957305, 2.0822562e-15, 0.00042692822], [1.0, 2.0028892e-24, 9.96402e-15], [1.0, 1.868542e-11, 3.6732317e-10], [0.052277833, 0.80998623, 0.137736], [0.9999995, 2.6909502e-15, 5.3224403e-07], [1.0, 6.367066e-23, 7.20034e-16], [1.0, 5.131369e-22, 2.049547e-19], [1.0, 2.0310508e-15, 4.2303107e-16], [1.0, 4.3655745e-25, 8.572204e-12], [1.0, 3.398265e-18, 2.097732e-10], [1.0, 5.6541905e-19, 2.1866869e-21], [1.0, 2.1213507e-21, 1.4229292e-24], [1.0, 2.8657532e-12, 4.1585846e-11], [0.9998472, 